In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path="configs", version_base="1.3")
cfg = compose(config_name="train")

In [ ]:
# Door dataset
toy_dataset = {
    "id": "door-full-new",
    "train-train": [
        "8877",
        "8893",
        "8897",
        "8903",
        "8919",
        "8930",
        "8961",
        "8997",
        "9016",
        "9032",
        "9035",
        "9041",
        "9065",
        "9070",
        "9107",
        "9117",
        "9127",
        "9128",
        "9148",
        "9164",
        "9168",
        "9277",
        "9280",
        "9281",
        "9288",
        "9386",
        "9388",
        "9410",
    ],
    "train-test": ["8867", "8983", "8994", "9003", "9263", "9393"],
    "test": ["8867", "8983", "8994", "9003", "9263", "9393"],
}
# special_req = "half-half"  # "fully-closed"
special_req = "half-half-01"

In [ ]:
from open_anything_diffusion.datasets.flow_trajectory import FlowTrajectoryDataModule
datamodule = FlowTrajectoryDataModule(
        root=cfg.dataset.data_dir,
        batch_size=cfg.training.batch_size,
        num_workers=cfg.resources.num_workers,
        n_proc=cfg.resources.n_proc_per_worker,
        seed=cfg.seed,
        history="his" in cfg.model.name,
        randomize_size=cfg.dataset.randomize_size,
        augmentation=cfg.dataset.augmentation,
        trajectory_len=1,  # Only used when training trajectory model
        special_req=special_req,  # special_req="fully-closed"
        n_repeat=200
        if special_req == "half-half-01"
        else (50 if special_req is None else 100),
        # # TODO: only for toy training!!!!!
        toy_dataset=toy_dataset,
    )

In [ ]:
dataloader = datamodule.train_val_dataloader(bsz=1)
samples = list(enumerate(dataloader))

In [ ]:
sample = samples[0][1]

In [ ]:
pcd = sample.pos
flow = sample.delta
history = sample.history
flow_history = sample.flow_history

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation

animation = FlowNetAnimation()

# sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
gt_action = torch.tensor([-1, 0, 0])
cosines = []
predictions = []
animation.add_trace(
    pcd,
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([pcd.numpy()]),
    torch.as_tensor([flow[:, 0, :].numpy() * 2]),
    # torch.as_tensor([pred_flow.cpu().numpy()]* 10),
    "red",
)

fig = animation.animate()
fig.show()

In [ ]:
import numpy as np
flip = 0 # 4 flip modes
flip_mat = torch.tensor([
                        [[1, 0, 0], [0, 1, 0], [0, 0, 1]],    # Normal
                        [[1, 0, 0], [0, -1, 0], [0, 0, 1]],    # Left, right
                        [[-1, 0, 0], [0, 1, 0], [0, 0, 1]],    # Front, back
                        [[-1, 0, 0], [0, -1, 0], [0, 0, 1]]     # Front back & left right
                    ]).float()

In [ ]:
animation = FlowNetAnimation()

for flip in range(4):
    # Add flip
    pcd = torch.matmul(sample.pos, flip_mat[flip])
    flow = torch.matmul(sample.delta, flip_mat[flip])
    history = torch.matmul(sample.history, flip_mat[flip])
    flow_history = torch.matmul(sample.flow_history, flip_mat[flip])

    # sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
    gt_action = torch.tensor([-1, 0, 0])
    cosines = []
    predictions = []
    animation.add_trace(
        pcd,
        # torch.as_tensor([pcd[mask]]),
        # torch.as_tensor([flow[mask]]),
        torch.as_tensor([pcd.numpy()]),
        torch.as_tensor([flow[:, 0, :].numpy() * 2]),
        # torch.as_tensor([pred_flow.cpu().numpy()]* 10),
        "red",
    )

fig = animation.animate()
fig.show()

## FOr real robot

In [ ]:
import numpy as np
path = '/home/yishu/open_anything_diffusion/real_world_pcds/slightly_open_xyz.npy'

import torch
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
## Point cloud only
animation = FlowNetAnimation()

# sampled_points = rot_pcd[np.random.randint(0, pcd.shape[0], 1200)]
gt_action = torch.tensor([-1, 0, 0])
cosines = []
predictions = []
animation.add_trace(
    torch.as_tensor(pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([pcd]),
    torch.as_tensor([np.zeros((1200, 3)) * 2]),
    # torch.as_tensor([pred_flow.cpu().numpy()]* 10),
    "red",
)


fig = animation.animate()
fig.show()

In [ ]:
prev_path = '/home/yishu/open_anything_diffusion/real_world_pcds/xyz.npy'
prev_pcd = np.load(prev_path)
# prev_rot_pcd = rotate_pcd(prev_pcd)

prev_flow_path = '/home/yishu/open_anything_diffusion/real_world_pcds/flow.npy'
history_flow = np.load(prev_flow_path)

import torch
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
## Point cloud only
animation = FlowNetAnimation()

animation.add_trace(
    torch.as_tensor(prev_pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([prev_pcd]),
    torch.as_tensor([history_flow]),
    # torch.as_tensor([pred_flow.cpu().numpy()]* 10),
    "red",
)

fig = animation.animate()
fig.show()
# prev_rot_pcd = rotate_pcd(prev_pcd)

prev_flow_path = '/home/yishu/open_anything_diffusion/real_world_pcds/flow.npy'
history_flow = np.load(prev_flow_path)

import torch
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
## Point cloud only
animation = FlowNetAnimation()

animation.add_trace(
    torch.as_tensor(prev_pcd),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([prev_pcd]),
    torch.as_tensor([history_flow]),
    # torch.as_tensor([pred_flow.cpu().numpy()]* 10),
    "red",
)

fig = animation.animate()
fig.show()